In [53]:
from __future__ import print_function
import collections
import pickle
import re
import random
import sys
import os 
import time

import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.fasttext import FastText
from gensim.models import word2vec

from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix,precision_recall_fscore_support
from sklearn import svm

import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from numpy import cumsum

import tensorflow as tf
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential,Model,load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout, Activation, Flatten, \
    Embedding, Convolution1D, MaxPooling1D, AveragePooling1D, \
    Input, Dense, merge, Add,TimeDistributed, Bidirectional,SpatialDropout1D, \
    Concatenate, concatenate, multiply, Lambda, dot, Layer
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.regularizers import l2, l1_l2
from keras.constraints import maxnorm
from keras import callbacks
from keras.utils import generic_utils,plot_model
from keras.optimizers import Adadelta,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
# from keras.engine.topology import Layer


import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

import sys
import re
import os
from pathlib import Path
from collections import namedtuple
import numpy as np
import pandas as pd
import pickle
import gensim.downloader as gensimapi
from collections import Counter, defaultdict
import ast

In [54]:
!ls

'Cross Lingual Aspect Based Sentiment Analysis- Part 1.ipynb'
'Cross Lingual Aspect Based Sentiment Analysis- Part 2.ipynb'
 keras2.ipynb
 Keras.ipynb
'Multilingual  semeval  dataset creation.ipynb'
 results
'Tatoeba Similarity Search.ipynb'
 Twitter_Data.ipynb
 utils


In [55]:
# data_folder = "../../../corpus/new/preprocess_from_isuru/"
# lankadeepa_data_path = data_folder + 'lankadeepa_tagged_comments.csv'
# gossip_lanka_data_path = data_folder + 'gossip_lanka_tagged_comments.csv'

# lankadeepa_data = pd.read_csv(lankadeepa_data_path)[:9059]
# gossipLanka_data = pd.read_csv(gossip_lanka_data_path)
# gossipLanka_data = gossipLanka_data.drop(columns=['Unnamed: 3'])

# sinhala_data = pd.concat([lankadeepa_data,gossipLanka_data], ignore_index=True)
# sinhala_data.to_csv("../data/sinhala_comments.csv")

In [56]:
train_aspects_ungrp = pd.read_csv('../data/Train_english_restaurants_ungrp.csv')
val_aspects_ungrp2 = pd.read_csv('../data/Valid_english_restaurants_ungrp.csv')
du_aspects_ungrp2  = pd.read_csv('../data/Dutch_restaurants_ungrp.csv')
sp_aspects_ungrp2  = pd.read_csv('../data/Spanish_english_restaurants_ungrp.csv')

sinhala_comments  = pd.read_csv('../data/sinhala_comments.csv')


In [57]:
sinhala_comments = sinhala_comments.loc[sinhala_comments['label'].isin([2, 3, 4])]

In [58]:
(unique, counts) = np.unique(sinhala_comments["label"].values, return_counts = True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   2 7665]
 [   3 2403]
 [   4 3080]]


In [59]:
train_aspects_ungrp[['text']].to_csv('../data/processed/en_resturant.csv' , header = None , index = None , mode = 'w')
val_aspects_ungrp2[['text']].to_csv('../data/processed/en_val.csv' , header = None , index = None , mode = 'w')
du_aspects_ungrp2[['text']].to_csv('../data/processed/nl_resturant.csv' , header = None , index = None , mode = 'w')
sp_aspects_ungrp2[['text']].to_csv('../data/processed/es_resturant.csv' , header = None , index = None , mode = 'w')

sinhala_comments[['comment']].to_csv('../data/processed/sinhala_comments.csv' , header = None , index = None , mode = 'w')


In [60]:
LASER_PATH = "/home/lahiru/Projects/FYP/LASER/LASER"
sys.path.insert(0, LASER_PATH + '/source/lib')
sys.path.insert(1, LASER_PATH + '/source')

DATA_PATH = Path("../data/processed/")
CACHE_PATH = Path("../cache2/")
CACHE_PATH.mkdir(exist_ok=True)
MODEL_PATH = Path(LASER_PATH + "/models")

os.environ["LASER"] = LASER_PATH 
SPACE_NORMALIZER = re.compile("\s+")
Batch = namedtuple('Batch', 'srcs tokens lengths')

In [61]:
from indexing import IndexLoad, IndexTextOpen, IndexTextQuery, IndexSearchKNN, IndexCreate, IndexSearchMultiple
from embed import SentenceEncoder, EncodeLoad, EncodeFile
from text_processing import Token, BPEfastApply

In [62]:
encoder = SentenceEncoder(
    str(MODEL_PATH / "bilstm.93langs.2018-12-26.pt"),
    max_sentences=None,
    max_tokens=10000,
    cpu=False)
#original : cpu = False

bpe_codes = str(MODEL_PATH / "93langs.fcodes")

for lang in ("en" ,"nl", 'es'): 
    Token(
        str(DATA_PATH / f"{lang}_resturant.csv"), ##english_resturant.txt
        str(CACHE_PATH / f"{lang}_resturant.csv"),
        lang=lang,
        romanize=False,
        lower_case=True, gzip=False,
        verbose=True)
    BPEfastApply(
        str(CACHE_PATH / f"{lang}_resturant.csv"),
        str(CACHE_PATH / f"{lang}_resturant.bpe"),
        bpe_codes,
        verbose=True, over_write=True)
    EncodeFile(
        encoder,
        str(CACHE_PATH / f"{lang}_resturant.bpe"),
        str(CACHE_PATH / f"{lang}_resturant.enc"),
        verbose=True, over_write=True)    
    
    
Token(
    str(DATA_PATH / f"en_val.csv"), ##english_resturant.txt
    str(CACHE_PATH / f"en_val.csv"),
    lang=lang,
    romanize=False,
    lower_case=True, gzip=False,
    verbose=True)
BPEfastApply(
    str(CACHE_PATH / f"en_val.csv"),
    str(CACHE_PATH / f"en_val.bpe"),
    bpe_codes,
    verbose=True, over_write=True)
EncodeFile(
    encoder,
    str(CACHE_PATH / f"en_val.bpe"),
    str(CACHE_PATH / f"en_val.enc"),
    verbose=True, over_write=True)    

 - Tokenizer: en_resturant.csv exists already
 - Tokenizer: nl_resturant.csv exists already
 - Tokenizer: es_resturant.csv exists already
 - Tokenizer: en_val.csv exists already


In [63]:
Token(
    str(DATA_PATH / f"sinhala_comments.csv"), ##english_resturant.txt
    str(CACHE_PATH / f"sinhala_comments.csv"),
    lang="sin",
    romanize=False,
    lower_case=True, gzip=False,
    verbose=True)
BPEfastApply(
    str(CACHE_PATH / f"sinhala_comments.csv"),
    str(CACHE_PATH / f"sinhala_comments.bpe"),
    bpe_codes,
    verbose=True, over_write=True)
EncodeFile(
    encoder,
    str(CACHE_PATH / f"sinhala_comments.bpe"),
    str(CACHE_PATH / f"sinhala_comments.enc"),
    verbose=True, over_write=True)  

 - Tokenizer: sinhala_comments.csv exists already


In [64]:
LASER_PATH = "/home/lahiru/Projects/FYP/LASER/LASER"
sys.path.insert(0, LASER_PATH + '/source/lib')
sys.path.insert(1, LASER_PATH + '/source')

DATA_PATH = Path("../data/processed/")
CACHE_PATH = Path("../cache2/")
CACHE_PATH.mkdir(exist_ok=True)
MODEL_PATH = Path(LASER_PATH + "/models")

os.environ["LASER"] = LASER_PATH 
SPACE_NORMALIZER = re.compile("\s+")
Batch = namedtuple('Batch', 'srcs tokens lengths')

In [65]:
train_en, index_tr_en = IndexCreate(
    str(CACHE_PATH / "en_resturant.enc"), 'FlatL2', verbose=True, save_index=False)
val_en, index_val_en = IndexCreate(
    str(CACHE_PATH / "en_val.enc"), 'FlatL2', verbose=True, save_index=False)

data_du, index_du = IndexCreate(
    str(CACHE_PATH / "nl_resturant.enc"), 'FlatL2', verbose=True, save_index=False)
data_spanish, index_spanish = IndexCreate(
    str(CACHE_PATH / "es_resturant.enc"), 'FlatL2', verbose=True, save_index=False)

data_sinhala, index_sinhala = IndexCreate(
    str(CACHE_PATH / "sinhala_comments.enc"), 'FlatL2', verbose=True, save_index=False)

 - embedding: ../cache2/en_resturant.enc 1656 examples of dim 1024
 - creating FAISS index
 - embedding: ../cache2/en_val.enc 283 examples of dim 1024
 - creating FAISS index
 - embedding: ../cache2/nl_resturant.enc 960 examples of dim 1024
 - creating FAISS index
 - embedding: ../cache2/es_resturant.enc 1416 examples of dim 1024
 - creating FAISS index
 - embedding: ../cache2/sinhala_comments.enc 13148 examples of dim 1024
 - creating FAISS index


In [66]:
def change_target(x):
    if x=='positive':
        return 2
    elif x =='negative':
        return 1
    else:
        return 0 
    
def change_target_sinala(x):
    if x== 4:
        return 2
    elif x == 2:
        return 1
    else:
        return 0 

train_aspects_ungrp['polarities'] = train_aspects_ungrp['polarities'].apply(lambda x: change_target(x))
val_aspects_ungrp2['polarities'] = val_aspects_ungrp2['polarities'].apply(lambda x: change_target(x))
sp_aspects_ungrp2['polarities'] = sp_aspects_ungrp2['polarities'].apply(lambda x: change_target(x))
du_aspects_ungrp2['polarities'] = du_aspects_ungrp2['polarities'].apply(lambda x: change_target(x))

sinhala_comments['label'] = sinhala_comments['label'].apply(lambda x: change_target_sinala(x))

In [67]:
from sklearn.preprocessing import MultiLabelBinarizer
tr_eng =  train_aspects_ungrp['polarities'].values
val_eng =val_aspects_ungrp2['polarities'].values  
y_du  = du_aspects_ungrp2['polarities'].values
y_spainish  = sp_aspects_ungrp2['polarities'].values

y_sinhala  = sinhala_comments['label'].values
tr_eng.shape , val_eng.shape , y_du.shape , y_spainish.shape, y_sinhala.shape

((1656,), (283,), (960,), (1416,), (13148,))

In [47]:
(unique, counts) = np.unique(y_sinhala, return_counts = True)
frequencies = np.asarray((unique, counts)).T
print(frequencies)

[[   0 2403]
 [   1 7665]
 [   2 3080]]


In [48]:
tr_eng = pd.get_dummies(tr_eng).values
val_eng = pd.get_dummies(val_eng).values
y_du = pd.get_dummies(y_du).values
y_spainish = pd.get_dummies(y_spainish).values
y_sinhala = pd.get_dummies(y_sinhala).values

In [49]:
tr_eng[0]

array([0, 0, 1], dtype=uint8)

In [68]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler().fit(train_en)
train_std = std_scale.transform(train_en) 
val_std = std_scale.transform(val_en)
dutch_std = std_scale.transform(data_du)
spanish_std = std_scale.transform(data_spanish)
sinhala_std = std_scale.transform(data_sinhala)

In [51]:
train_std.shape

(1656, 1024)

In [21]:
time_steps = 64
features = int(len(train_std[0]))//64

In [22]:
train_std = train_std.reshape(len(train_std), time_steps, features)
sinhala_std = sinhala_std.reshape(len(sinhala_std), time_steps, features)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(sinhala_std, y_sinhala, test_size=0.2, random_state=0)

In [24]:
train_std.shape

(1656, 64, 16)

In [25]:
def RNN_model():
    main_input = Input(shape=(time_steps,features ), dtype='float', name='main_input')
#     embedding  = Embedding(MAX_FEATURES, EMBEDDING_SIZE
#               , input_length=MAX_LEN,
#               name='embedding' ,trainable=False)(main_input)

#     embedding = Dropout(DROPOUT_VALUE_1)(main_input)

    x = RNN(HIDDEN_DIMS)(main_input)

    x = Dense(HIDDEN_DIMS, activation='relu', init='he_normal', 
              W_constraint = maxnorm(3), b_constraint=maxnorm(3),
              name='mlp')(x)

    x = Dropout(DROPOUT_VALUE_2, name='drop')(x)

    output = Dense(3, init='he_normal',
                   activation='softmax', name='output')(x)

    model = Model(input=main_input, output=output ,name="RNN_model")

    model.compile(loss={'output':'categorical_crossentropy'},
              optimizer=Adadelta(lr=0.95, epsilon=1e-06),
              metrics=["accuracy",
                       keras.metrics.Precision(),
                        keras.metrics.Recall(),
                       f1])
    
    print(model.summary())
    return model

# def simple_model():
#     model = tf.keras.Sequential()
#     model.add(tf.keras.layers.Dense(512))
#     model.add(tf.keras.layers.Dense(256))
#     model.add(tf.keras.layers.Dense(3))
#     model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=["accuracy"])
#     # This builds the model for the first time:
    
#     return model

# def simple_model2():
    
#     model = Sequential()
#     model.add(LSTM(32, input_shape=(1024, 1024)))
#     model.add(Dense(3))
#     print(model.summary())
#     model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=["accuracy"])
#     return model

    


In [26]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
def Train_Model(model,X_train, y_train, validation_data , cross_validation = False):
    print('Training and Testing...')
    es = EarlyStopping(monitor='val_f1', mode='max', verbose=1, patience=5)
    checkpoint = ModelCheckpoint(model_save_path, monitor='val_f1', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,es]
    
    if (cross_validation):
        callbacks_list = [es]
        
    his = model.fit(X_train, y_train,validation_data = validation_data , epochs=NB_EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    #his = model.fit(X_train, y_train,validation_data = validation_data , epochs=NB_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list, verbose=1)
    return model,his

In [28]:
MAX_FEATURES = 300
VERBOSITY = 1
VALIDATION_SPLIT = 0.1
NB_EPOCHS = 50
FOLDS = 10
EMBEDDING_SIZE = 300

BATCH_SIZE = 32 # 64, 128
NB_FILTERS = 200 #200
FILTER_LENGTH = 4 # test with 2,3,4,5
# HIDDEN_DIMS = NB_FILTERS * 2
HIDDEN_DIMS = 32
MAX_LEN = 210 #test with other values(only this value work for now)
DROPOUT_VALUE_1 = 0.5 #0.8 #0.3
DROPOUT_VALUE_2 = 0.5
L2_REG= 0.01

RNN = LSTM

model_save_path = "../trained_models/RNN1.hdf5"

In [29]:
model = RNN_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "RNN_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 64, 16)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6272      
_________________________________________________________________
mlp (Dense)                  (None, 32)                1056      
_________________________________________________________________
drop (Dropout)               (None, 32)                0         
_________________________________________________________________
output (Dense)               (None, 3)                 99        
Total params: 7,427
Trainable params: 7,427
Non-trainable params: 0
____________

/home/lahiru/anaconda3/envs/LASER-tf-1/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", name="mlp", kernel_initializer="he_normal", kernel_constraint=<keras.con..., bias_constraint=<keras.con...)`
  del sys.path[0]
/home/lahiru/anaconda3/envs/LASER-tf-1/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softmax", name="output", kernel_initializer="he_normal")`
/home/lahiru/anaconda3/envs/LASER-tf-1/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="RNN_model", inputs=Tensor("ma..., outputs=Tensor("ou...)`


In [ ]:
Train_Model(model,train_std, tr_eng,validation_data = (X_test, y_test), cross_validation = False)

Training and Testing...
Train on 1656 samples, validate on 2630 samples
Epoch 1/50
1656/1656 [==============================] - 3s 2ms/step - loss: 0.8678 - accuracy: 0.6141 - precision_4: 0.6831 - recall_4: 0.4022 - f1: 0.4726 - val_loss: 1.1203 - val_accuracy: 0.3567 - val_precision_4: 0.3416 - val_recall_4: 0.2141 - val_f1: 0.2642
Epoch 2/50
1656/1656 [==============================] - 2s 1ms/step - loss: 0.7436 - accuracy: 0.6727 - precision_4: 0.7071 - recall_4: 0.5990 - f1: 0.6465 - val_loss: 1.1127 - val_accuracy: 0.4681 - val_precision_4: 0.4768 - val_recall_4: 0.3821 - val_f1: 0.4244
Epoch 3/50
1656/1656 [==============================] - 1s 707us/step - loss: 0.7084 - accuracy: 0.7047 - precision_4: 0.7283 - recall_4: 0.6377 - f1: 0.6805 - val_loss: 1.4039 - val_accuracy: 0.2783 - val_precision_4: 0.2698 - val_recall_4: 0.2490 - val_f1: 0.2612
Epoch 4/50
1656/1656 [==============================] - 1s 799us/step - loss: 0.6780 - accuracy: 0.7156 - precision_4: 0.7397 - recall

In [81]:
X_train, X_test, y_train, y_test = train_test_split(sinhala_std, y_sinhala, test_size=0.2, random_state=0)

In [82]:
clf = svm.SVC(decision_function_shape='ovo')

In [83]:
clf.fit(train_std, tr_eng)

SVC(decision_function_shape='ovo')

In [85]:
prediction = clf.predict(X_test)

In [88]:
report_print = classification_report(y_test, prediction, digits=4)
print(report_print)

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       453
           1     0.7725    0.5251    0.6252      1571
           2     0.3412    0.8795    0.4917       606

    accuracy                         0.5163      2630
   macro avg     0.3712    0.4682    0.3723      2630
weighted avg     0.5401    0.5163    0.4868      2630



/home/lahiru/anaconda3/envs/LASER-tf-1/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
